In [1]:
# library
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import r2_score

torch.manual_seed(1)
np.random.seed(1)
torch.set_printoptions(precision=3)

In [2]:
# Model
class Net(torch.nn.Module):
    def __init__(self, s):
        super(Net, self).__init__()
        self.input   = torch.nn.Linear(3, s)
        self.output  = torch.nn.Linear(s, 1)

    def forward(self, x):
        z = torch.tanh(self.input(x))
        z = self.output(z)
        return z

In [3]:
# Model error
def eval(model, testset, p):
    with torch.no_grad():
        pred_Y = model(testset.x_data)
    
    # R^2 qrh
    r2_qrh = r2_score(testset.y_data, pred_Y)

    # R^2 msa
    pred_msa = pred_Y*p[0] + p[1]
    r2_msa = r2_score(testset.msa_data, pred_msa)
    
    return r2_qrh, r2_msa

In [4]:
# Data
class Data(torch.utils.data.Dataset):
  def __init__(self, src_file, start=None, end=None):
    df = pd.read_csv(src_file)
    clg_sp       = np.array(df['clg_sp_current']).reshape(-1,1)[start: end]
    htg_sp       = np.array(df['htg_sp_current']).reshape(-1,1)[start: end]
    htg_clg_mode = 1*np.array(df['htg_clg_mode']).reshape(-1,1)[start: end]

    sp_k   = htg_sp*htg_clg_mode + clg_sp*(1-htg_clg_mode)
    Tz_k   = np.array(df['thermostat_room_temp']).reshape(-1,1)[start: end]
    qrh_k  = np.array(df['htg_valve_command']).reshape(-1,1)[start: end]
    qrh_k1 = np.array(df['htg_valve_command']).reshape(-1,1)[start+1: end+1]
    msa_k1 = np.array(df['airflow_desired']).reshape(-1,1)[start+1: end+1]
    tmp_x  = np.concatenate((sp_k, Tz_k, qrh_k), axis=1)
    
    self.x_data   = torch.tensor(tmp_x,  dtype=torch.float32)
    self.y_data   = torch.tensor(qrh_k1, dtype=torch.float32)
    self.msa_data = torch.tensor(msa_k1, dtype=torch.float32)

  def __len__(self):
    return len(self.x_data)

  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()
    inp  = self.x_data[idx]
    outp = self.y_data[idx]
    msa  = self.msa_data[idx]
    sample = {'inp':inp, 'outp':outp, 'msa':msa}
    return sample

In [5]:
# Early stopping
def early_stop(list, min_epochs, patience):
    if(len(list) > min_epochs):
        if(np.max(list[-patience:]) < 1.0001*np.max(list[0: -patience])):
            return 1
    return 0

In [6]:
# train function
def train(net, p, train_ds, test_ds, lr=0.001, min_epochs=200, max_epochs=100000, patience=100, smooth=0, soft=0):
    
    loss_func  = torch.nn.MSELoss()
    optimizer  = torch.optim.Adam(net.parameters(), lr=lr)

    train_ldr = torch.utils.data.DataLoader(train_ds, batch_size=256, shuffle=True)
    R2_qrh = np.array([])
    R2_msa = np.array([])
    for _ in range(0, max_epochs+1):
        net.train()
        loss  = 0
        count = 0
        for (_, batch) in enumerate(train_ldr):
            X    = batch['inp']
            Y    = batch['outp']

            optimizer.zero_grad()
            output = net(X)                    # compute the output of the Network
            loss_val = loss_func(output, Y) + smooth*loss_func(output, X[:,2].reshape(-1,1)) + soft*torch.sum(torch.square(torch.relu(0-output))) + soft*torch.sum(torch.square(torch.relu(output-100)))
            loss += loss_val.item()            # accumulate
            loss_val.backward()                # gradients
            optimizer.step()                   # update paramters
            count += 1

        net.eval()
        R2_qrh = np.append(R2_qrh, eval(net, test_ds, p)[0].item())
        R2_msa = np.append(R2_msa, eval(net, test_ds, p)[1].item())
        
        if(early_stop(list = R2_qrh, min_epochs = min_epochs, patience = patience) == 1):
            break
    
    return R2_qrh, R2_msa

In [7]:
# main

for n_train in [32, 64, 128]:
    for _smooth in [0, 0.001, 0.01]:
        for _soft in [0, 1e3]:
            for h in [16, 32, 64]:
                for _lr in [0.0001, 0.001, 0.01, 0.1]:

                    # Read data
                    df = pd.read_csv('C:/Users/tln229/Downloads/Python/1. Building/data/HVAC_B90_102_exp_10m_20210424.csv')
                    qrh = np.array(df['htg_valve_command']).reshape(-1,1)
                    msa = np.array(df['airflow_desired']).reshape(-1,1)

                    # LEAST SQUARE
                    ones = np.ones(msa.shape)
                    A = np.concatenate((qrh, ones), axis=1)
                    b = np.copy(msa)
                    p = np.linalg.lstsq(A, b, rcond=None)[0]

                    # Create network
                    device = torch.device("cpu")
                    net = Net(h).to(device)

                    # Create Dataset and DataLoader objects
                    src_file = 'C:/Users/tln229/Downloads/Python/1. Building/data/HVAC_B90_102_exp_10m_20210424.csv'
                    train_ds = Data(src_file, start=0, end=n_train)
                    test_ds  = Data(src_file, start=n_train, end=1600)

                    # train
                    R2_qrh, R2_msa = train(net, p, train_ds, test_ds, lr=_lr, min_epochs=500, max_epochs=100000, patience=300, smooth=_smooth, soft=_soft)

                    # results
                    print('n train = %3d \t smooth = %6.4f \t layer size = %2d \t lr = %6.4f \t best_epoch = %5d \t best_R2_qrh = %7.5f \t best_R2_msa = %7.5f'
                        % (n_train, _smooth, h, _lr, np.argmax(R2_qrh), np.max(R2_qrh), np.max(R2_msa)))

n train =  32 	 smooth = 0.0000 	 layer size = 16 	 lr = 0.0001 	 best_epoch = 55782 	 best_R2_qrh = 0.81091 	 best_R2_msa = 0.80059
n train =  32 	 smooth = 0.0000 	 layer size = 16 	 lr = 0.0010 	 best_epoch = 21691 	 best_R2_qrh = 0.66023 	 best_R2_msa = 0.65769
n train =  32 	 smooth = 0.0000 	 layer size = 16 	 lr = 0.0100 	 best_epoch =  4146 	 best_R2_qrh = 0.75182 	 best_R2_msa = 0.74337
n train =  32 	 smooth = 0.0000 	 layer size = 16 	 lr = 0.1000 	 best_epoch =  1114 	 best_R2_qrh = 0.53840 	 best_R2_msa = 0.53358
n train =  32 	 smooth = 0.0000 	 layer size = 32 	 lr = 0.0001 	 best_epoch = 61416 	 best_R2_qrh = 0.82459 	 best_R2_msa = 0.81266
n train =  32 	 smooth = 0.0000 	 layer size = 32 	 lr = 0.0010 	 best_epoch = 12058 	 best_R2_qrh = 0.77628 	 best_R2_msa = 0.76626
n train =  32 	 smooth = 0.0000 	 layer size = 32 	 lr = 0.0100 	 best_epoch =  4421 	 best_R2_qrh = 0.73567 	 best_R2_msa = 0.72766
n train =  32 	 smooth = 0.0000 	 layer size = 32 	 lr = 0.1000 	 bes